In [12]:
!pip install node2vec

In [4]:
import pandas as pd
import networkx as nx

filtered_df = pd.read_csv('interactions_human_reduced.csv')
df = filtered_df[['gene_id','interactant_id']].drop_duplicates().sort_values(by=['gene_id','interactant_id'])
graph_df = df.loc[df['gene_id'] != df['interactant_id']]

# between_df = pd.read_csv('drive/MyDrive/definitivo/betweenness_centrality.csv')
# close_df = pd.read_csv('drive/MyDrive/definitivo/closeness_centrality.csv')
# centrality_df = pd.merge(between_df, close_df, on='Node', how='inner')

G = nx.from_pandas_edgelist(graph_df, source='gene_id', target='interactant_id')

In [21]:
from node2vec import Node2Vec

try:
    print("Starting Node2Vec")
    !free -h
    # node2vec = Node2Vec(G, dimensions=64, walk_length=20, num_walks=40, p=2, q=0.5, workers=1)
    # node2vec = Node2Vec(G, dimensions=64, walk_length=20, num_walks=40, p=2, q=0.5, workers=1)
    node2vec = Node2Vec(G, dimensions=64, walk_length=20, num_walks=40, p=2, q=0.5, workers=1)
    !free -h
    print("Node2Vec initialization complete, starting training")
    model = node2vec.fit(window=15, min_count=1, batch_words=4)
    print("Training complete")

    # Obtenemos los embeddings del modelo
    embeddings = model.wv

    # Ejemplo de obtención del embedding para un nodo concreto
    node_id_example = list(G.nodes())[0]
    print(f"Embedding for node {node_id_example}: {embeddings[node_id_example]}")
    
except Exception as e:
    print(f"Ocurrió un error: {e}")


Starting Node2Vec
               total        used        free      shared  buff/cache   available
Mem:            50Gi        10Gi        34Gi       2.0Mi       5.9Gi        40Gi
Swap:             0B          0B          0B


Computing transition probabilities:   0%|          | 0/19271 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 40/40 [09:13<00:00, 13.83s/it]

               total        used        free      shared  buff/cache   available
Mem:            50Gi        15Gi        29Gi       2.0Mi       6.0Gi        35Gi
Swap:             0B          0B          0B


Node2Vec initialization complete, starting training
Training complete
Embedding for node 1: [ 0.30836555 -0.20123638  0.01891314  0.20398627 -0.02763476 -0.24290173
  0.14790314 -0.03796874 -0.17933637 -0.01624922  0.15941891 -0.22617191
 -0.1069317   0.00848343  0.04340406 -0.02778855 -0.17753956 -0.12936647
 -0.09139332  0.02774772  0.09093016  0.16738677  0.08445685 -0.0093413
 -0.15658216  0.12747869 -0.15662028  0.07476033 -0.03843369  0.27683353
 -0.05931099  0.03155842 -0.06388857 -0.08558691  0.18491463 -0.0648582
  0.01019941 -0.01642682  0.07329363 -0.07475763  0.16557649  0.14848989
  0.2163282  -0.24588156  0.15889658 -0.12847744  0.06339542  0.07681213
  0.11805064  0.09591445  0.03590894  0.05379218  0.0819378   0.10633777
  0.22349809  0.14655662  0.0699714   0.04893305 -0.00793832  0.1347617
 -0.1200747  -0.13673505 -0.10902493  0.15403819]


In [23]:
node_labels = list(G.nodes)
embedding_dict = {node_labels[i]: embeddings[i] for i in range(len(node_labels))}

embedding_df = pd.DataFrame(embedding_dict)
embedding_df.to_csv('drive/MyDrive/definitivo/node2vec_embeddings5.csv')